# **Install OpenCV Libraries** 


In [2]:
!pip uninstall opencv-python -y
# downgrade OpenCV a bit since some none-free features are not available
#!pip install opencv-contrib-python==3.4.2.17 --force-reinstall
%cd /content/drive/My Drive/Unimore/VCS/Project/OpenCV
!pip install opencv_contrib_python_headless-4.1.0.25-cp36-cp36m-linux_x86_64.whl --force-reinstall

Uninstalling opencv-python-4.1.2.30:
  Successfully uninstalled opencv-python-4.1.2.30
/content/drive/My Drive/Unimore/VCS/Project/OpenCV
Processing ./opencv_contrib_python_headless-4.1.0.25-cp36-cp36m-linux_x86_64.whl
     |████████████████████████████████| 20.2MB 5.4MB/s 
ERROR: imgaug 0.2.9 requires opencv-python, which is not installed.
ERROR: dopamine-rl 1.0.5 requires opencv-python>=3.4.1.15, which is not installed.
ERROR: albumentations 0.1.12 requires opencv-python, which is not installed.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.4
    Uninstalling numpy-1.18.4:
      Successfully uninstalled numpy-1.18.4


# **Root Path Definition** #

In [0]:
root_path = "/content/drive/My Drive/Unimore/VCS/Project/"

# **Convert .MOV to .MP4**

In [2]:
import subprocess
import os
import re
import subprocess as sp
import logging

the_file = root_path + "Videos/IMG_4081.MOV"
out_file = root_path + "Videos/IMG_4081.MP4"

ffmpeg = sp.Popen(['/usr/bin/ffmpeg', '-i', the_file, out_file], stdout = sp.PIPE, stderr = sp.STDOUT)
process_output =  ffmpeg.communicate()
print ("communicate: " + str(process_output)) 

communicate: (b'ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers\n  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)\n  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --en

# **Import Mask-RCNN Paths**

In [0]:
import time
import random
import numpy as np


labelsPath = root_path + "mask-rcnn-coco/object_detection_classes_coco.txt" 
LABELS = open(labelsPath).read().strip().split("\n")


colorsPath = root_path + "mask-rcnn-coco/colors.txt" 
COLORS = open(colorsPath).read().strip().split("\n")
COLORS = [np.array(c.split(",")).astype("int") for c in COLORS]
COLORS = np.array(COLORS, dtype="uint8")

weightsPath = root_path + "mask-rcnn-coco/frozen_inference_graph.pb" 
configPath  = root_path + "mask-rcnn-coco/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt" 

# **Retrieve paintings names**

In [0]:
import pandas as pd

db = pd.read_csv(root_path + 'data.csv', sep = ',')
df = pd.DataFrame(db.values, columns = ['Title','Author','Room','Image']); 

Title = df.iloc[:,0].values
Author = df.iloc[:,1].values
Room = df.iloc[:,2].values
Image = df.iloc[:,3].values

DB_title = df.set_index('Image')['Title'].to_dict()
DB_room  = df.set_index('Image')['Room'].to_dict()

# **Create features DB file**

In [0]:
import pickle as pickle
import cv2
import os
import numpy as np

def extract_sift_features(image_path, sift, nfeatures, vector_size=128):

    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    if type(image) is np.ndarray:
        try:            
            kp, des = sift.detectAndCompute(image, None)
            # Making descriptor of same size
            needed_size = (vector_size * nfeatures)
            if des.size < needed_size:
                # if we have less the nfeatures descriptors then just adding zeros at the
                # end of our feature vector
                pad = (nfeatures - des.shape[0])                
                des = np.pad(des, ((0,pad),(0,0)), 'constant', constant_values=(0))                      
        except cv2.error as e:
            print ('Error: ' + srt(e))
            return None
        return des
    else:
      return None  


def batch_extractor(images_path, pickled_db_path, sift):

    files = [os.path.join(images_path, p) for p in sorted(os.listdir(images_path))]

    result = {}
    for f in files:
        head, tail = os.path.split(f)
        #print ('Extracting features from image ' + str(tail))        
        result[tail] = extract_sift_features(f, sift, nfeatures)
    
    # saving all our feature vectors in pickled file
    with open(pickled_db_path, 'wb') as fp:
        pickle.dump(result, fp)

In [0]:
images_path = root_path + 'paintings_db/'
pickled_db_path = root_path + 'paintings_db/features.pck'

nfeatures = 500
sift =  cv2.xfeatures2d.SIFT_create(nfeatures) 

#batch_extractor(images_path,pickled_db_path, sift) #uncomment this line to extract sift

with open(pickled_db_path, 'rb') as fp:
  data = pickle.load(fp)
  names = []
  matrix = []
  for k, v in data.items():
      names.append(k)
      matrix.append(v)
  matrix_db = np.array(matrix)
  names_db = np.array(names) 


# **HOG - SVM**

In [0]:
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    from skimage.feature import hog
    if vis == True:
        # Use skimage.hog() to get both features and a visualization
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell), cells_per_block=(cell_per_block, cell_per_block), visualize=vis, feature_vector=feature_vec)
        return features, hog_image
    else:      
        # Use skimage.hog() to get features only
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell), cells_per_block=(cell_per_block, cell_per_block), visualize=vis, feature_vector=feature_vec)
        return features

In [0]:
def extract_HOG_features(imgs, cspace='RGB', orient=9, pix_per_cell=8, cell_per_block=2, hog_channel=0, dim_img = 64, aug_feat = 6):
    
    import imageio
    import imgaug as ia
    import imgaug.augmenters as iaa
    
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in os.listdir(imgs):

        if str(file) != '.ipynb_checkpoints':
            # Read in each one by one
            image = cv2.imread(imgs + "/" + file)
            for i in range(aug_feat):
              if i == 0:
                image = image
              elif i == 1:
                rotate=iaa.Affine(rotate=(-30, 30))
                image=rotate.augment_image(image)    
              elif i == 2:
                gaussian_noise=iaa.AdditiveGaussianNoise(10,20)
                image=gaussian_noise.augment_image(image)
              elif i == 3:
                flip_hr=iaa.Fliplr(p=0.7)
                image= flip_hr.augment_image(image) 
              elif i == 4: 
                shear = iaa.Affine(shear=(0,10))
                image=shear.augment_image(image)
              elif i == 5:  
                crop = iaa.Crop(percent=(0, 0.3)) # crop image
                image=crop.augment_image(image)
              image = cv2.resize(image, (dim_img,dim_img), interpolation = cv2.INTER_AREA)       
              # apply color conversion if other than 'RGB'
              if cspace != 'RGB' and cspace != 'GRAY':
                  if cspace == 'HSV':
                      feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
                  elif cspace == 'LUV':
                      feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
                  elif cspace == 'HLS':
                      feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
                  elif cspace == 'Lab':
                      feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2Lab)
                  elif cspace == 'YUV':
                      feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
                  elif cspace == 'YCrCb':
                      feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
              else:
                if cspace == 'GRAY':
                  feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
                else:  
                  feature_image = np.copy(image)      

              # Call get_hog_features() with vis=False, feature_vec=True
              if hog_channel == 'ALL':
                  hog_features = []
                  if cspace != 'GRAY':
                    for channel in range(feature_image.shape[2]):
                        hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                            orient, pix_per_cell, cell_per_block, 
                                            vis=False, feature_vec=True))                    
                  else:
                        hog_features.append(get_hog_features(feature_image, 
                                            orient, pix_per_cell, cell_per_block, 
                                            vis=False, feature_vec=True))   
                  hog_features = np.ravel(hog_features)        
              else:
                if cspace != 'GRAY':
                  hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                              pix_per_cell, cell_per_block, vis=False, feature_vec=True)
                else:
                  hog_features = get_hog_features(feature_image, orient, 
                              pix_per_cell, cell_per_block, vis=False, feature_vec=True)  
              # Append the new feature vector to the features list
              features.append(hog_features)
    # Return list of feature vectors
    return features

In [0]:
def create_painting_classifier(path):
  # Extraction Parameters
  import time
  from sklearn.model_selection import train_test_split  
  from sklearn.svm import LinearSVC, SVC
  from sklearn.calibration import CalibratedClassifierCV  

  colorspace = 'Lab' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb, GRAY
  orient = 9
  pix_per_cell = 8
  cell_per_block = 2
  hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
  dim_img = 128
  aug_feat = 3

  t = time.time()

  painting_features = extract_HOG_features(path + '/Painting', cspace=colorspace, orient=orient,pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel, dim_img = dim_img, aug_feat = aug_feat)
  not_painting_features = extract_HOG_features(path + '/Not Painting', cspace=colorspace, orient=orient, pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel, dim_img = dim_img, aug_feat = aug_feat)

  t2 = time.time()
  print(round(t2-t, 2), 'Seconds to extract HOG features...')


  # Create an array stack of feature vectors
  X = np.vstack((painting_features, not_painting_features)).astype(np.float64)  

  # Define the labels vector
  y = np.hstack((np.ones(len(painting_features)), np.zeros(len(not_painting_features))))


  # Split up data into randomized training and test sets
  rand_state = np.random.randint(0, 100)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

  print('Using:',orient,'orientations',pix_per_cell,
      'pixels per cell and', cell_per_block,'cells per block')
  print('Feature vector length:', len(X_train[0]))

  # Use a linear SVC 
  model = CalibratedClassifierCV(base_estimator=LinearSVC(penalty='l2', dual=False), cv=5)
  # Check the training time for the SVC
  t = time.time()
  model.fit(X_train, y_train)
  t2 = time.time()
  print(round(t2-t, 2), 'Seconds to train SVC...')
  # Check the score of the SVC
  print('Test Accuracy of SVC = ', round(model.score(X_test, y_test), 4))
  # Check the prediction time for a single sample
  t=time.time()
  n_predict = 10
  print('My SVC predicts:     ', model.predict_proba(X_test[0:n_predict]))
  print('For these',n_predict, 'labels: ', y_test[0:n_predict])
  t2 = time.time()
  print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

  return model

In [0]:
#paint_model = create_painting_classifier(root_path + 'Dataset')

In [0]:
# Saving paint_model objects:
#with open(root_path + 'Dataset/SVC Model/paint_model.pkl', 'wb') as f:  
    #pickle.dump(paint_model, f)

In [0]:
# Getting back paint_model:
with open(root_path + 'Dataset/SVC Model/paint_model.pkl', 'rb') as f: 
    paint_model = pickle.load(f)

In [0]:
def create_people_statue_classifier(path):

  # Extraction Parameters
  import time
  from sklearn.model_selection import train_test_split  
  from sklearn.svm import LinearSVC
  from sklearn.calibration import CalibratedClassifierCV

  colorspace = 'HLS' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
  orient = 44
  pix_per_cell = 8
  cell_per_block = 1
  hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
  dim_img = 100
  aug_feat = 1

  t = time.time()

  people_features = extract_HOG_features(path + '/People', cspace=colorspace, orient=orient,pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel, dim_img = dim_img, aug_feat = aug_feat)
  statues_features = extract_HOG_features(path + '/Statues', cspace=colorspace, orient=orient, pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel, dim_img = dim_img, aug_feat = aug_feat)

  t2 = time.time()
  print(round(t2-t, 2), 'Seconds to extract HOG features...')


  # Create an array stack of feature vectors
  X = np.vstack((people_features, statues_features)).astype(np.float64)  

  # Define the labels vector
  y = np.hstack((np.ones(len(people_features)), np.zeros(len(statues_features))))
 

  # Split up data into randomized training and test sets
  rand_state = np.random.randint(0, 100)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

  print('Using:',orient,'orientations',pix_per_cell,
      'pixels per cell and', cell_per_block,'cells per block')
  print('Feature vector length:', len(X_train[0]))


  # Use a linear SVC 
  people_statue_model = CalibratedClassifierCV(base_estimator=LinearSVC(penalty='l2', dual=False), cv=5)
  # Check the training time for the SVC
  t = time.time()
  people_statue_model.fit(X_train, y_train)
  t2 = time.time()
  print(round(t2-t, 2), 'Seconds to train SVC...')
  # Check the score of the SVC
  print('Test Accuracy of SVC = ', round(people_statue_model.score(X_test, y_test), 4))
  # Check the prediction time for a single sample
  t=time.time()
  n_predict = 10
  print('My SVC predicts:     ', people_statue_model.predict_proba(X_test[0:n_predict]))
  print('For these',n_predict, 'labels: ', y_test[0:n_predict])
  t2 = time.time()
  print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')


  return people_statue_model

In [0]:
#people_statue_model = create_people_statue_classifier(root_path + 'Dataset')

In [0]:
# Saving people_statue_model objects:
#with open(root_path + 'Dataset/SVC Model/people_statue_model.pkl', 'wb') as f:  
    #pickle.dump(people_statue_model, f)

In [0]:
# Getting back paint_model:
with open(root_path + 'Dataset/SVC Model/people_statue_model.pkl', 'rb') as f: 
    people_statue_model = pickle.load(f)

# **Delete Images from folder**

In [0]:
import os, shutil
folder = '/content/drive/My Drive/Unimore/VCS/Project/Videos/Images' 

for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

# **Import**

In [0]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
from scipy.spatial import distance as dist

# **Geometric Functions**

In [0]:
def retrieve_points_coordinates(pts, x_offset= 0, y_offset = 0):

  rect = np.zeros((4, 2), dtype="int32")
  #tl
  rect[0] = (pts[0],pts[1])
  rect[0,[0]] += x_offset
  rect[0,[1]] += y_offset
  #tr
  rect[1] = (pts[0]+pts[2],pts[1])
  rect[1,[0]] += x_offset
  rect[1,[1]] += y_offset
  #bl
  rect[2] = (pts[0]+pts[2],pts[1]+pts[3])
  rect[2,[0]] += x_offset
  rect[2,[1]] += y_offset
  #br
  rect[3] = (pts[0],pts[1]+pts[3])
  rect[3,[0]] += x_offset
  rect[3,[1]] += y_offset
  
  return rect

In [0]:
def order_points(pts, x_offset= 0, y_offset = 0):
  
    rect = np.zeros((4, 2), dtype="int32")

    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]     
    rect[0,[0]] += x_offset
    rect[0,[1]] += y_offset
    rect[2] = pts[np.argmax(s)]     
    rect[2,[0]] += x_offset
    rect[2,[1]] += y_offset

    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[1,[0]] += x_offset
    rect[1,[1]] += y_offset
    rect[3] = pts[np.argmax(diff)]
    rect[3,[0]] += x_offset
    rect[3,[1]] += y_offset
    # return the ordered coordinates
    return rect 

In [0]:
def order_points_warp(pts, x_offset= 0, y_offset = 0):

    rect = np.zeros((4, 2), dtype="float32")

    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)] 
    rect[0,[0]] += x_offset
    rect[0,[1]] += y_offset
    rect[2] = pts[np.argmax(s)]  
    rect[2,[0]] += x_offset
    rect[2,[1]] += y_offset

    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[1,[0]] += x_offset
    rect[1,[1]] += y_offset
    rect[3] = pts[np.argmax(diff)]
    rect[3,[0]] += x_offset
    rect[3,[1]] += y_offset
    # return the ordered coordinates
    return rect

In [0]:
def order_points_by_clusters(pts, median, x_offset= 0, y_offset = 0):
  
  rect = np.zeros((4, 2), dtype="int32")

  west_cluster = []
  east_cluster = []
  
  for pt in pts:
    if pt[0] < int(median - x_offset):
      west_cluster.append(pt)
    else:
      east_cluster.append(pt)  

  west_cluster = np.array(west_cluster)
  west_cluster = west_cluster.reshape(-2,2)
  east_cluster = np.array(east_cluster)
  east_cluster = east_cluster.reshape(-2,2)


  #print(pts)
  #print(median)
  #print(west_cluster)
  #print(east_cluster)

  #print(west_cluster,east_cluster)
  [tl_west,tr_west,br_west,bl_west] = order_points(west_cluster)  
  [tl_east,tr_east,br_east,bl_east] = order_points(east_cluster)

  rect[0] = tl_west
  rect[0,[0]] += x_offset
  rect[0,[1]] += y_offset
  rect[1] = tr_east
  rect[1,[0]] += x_offset
  rect[1,[1]] += y_offset 
  rect[2] = br_east
  rect[2,[0]] += x_offset
  rect[2,[1]] += y_offset 
  rect[3] = bl_west
  rect[3,[0]] += x_offset
  rect[3,[1]] += y_offset 

  return rect
 

In [0]:
def sum_offset_coordinates(coordinates, x_offset= 0, y_offset = 0):
  
  for i in range(len(coordinates)):
    coordinates[i,[0]] += x_offset
    coordinates[i,[1]] += y_offset

  return coordinates

# **Perspective Transform**

In [0]:
def perspective_transform(image, pts):

    rect = np.zeros((4, 2), dtype="float32")

    rect = order_points_warp(pts)
    (tl, tr, br, bl) = pts

    width_A = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    width_B = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    max_width = max(int(width_A), int(width_B))

    height_A = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    height_B = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    max_height = max(int(height_A), int(height_B))

    dst = np.array([
      [0, 0],
      [max_width - 1, 0],
      [max_width - 1, max_height - 1],
      [0, max_height - 1]], dtype = "float32")
    
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (max_width, max_height))
    
    return warped

# **Binary Image**

In [0]:
def transform_image(img): 

  gray_img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)           
  element = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
  closing = cv2.morphologyEx(gray_img, cv2.MORPH_GRADIENT, element)
  morph_simple_img = cv2.morphologyEx(closing, cv2.MORPH_CLOSE, element)
  morph_simple_img = cv2.cvtColor(morph_simple_img,cv2.COLOR_GRAY2RGB)

  #DEBUG
  #plt.figure(figsize = (20,10))
  #plt.imshow(morph_simple_img, cmap=plt.cm.gray, interpolation='nearest')

  black_detect = img.copy()
  mask = cv2.compare(black_detect,65,cv2.CMP_LT)
  black_detect[mask > 0] = 255
  black_detect = cv2.cvtColor(black_detect,cv2.COLOR_RGB2GRAY)
  black_detect = cv2.GaussianBlur(black_detect,(5,5),0)
  _, black_detect = cv2.threshold(black_detect, 240, 255, cv2.THRESH_BINARY)  
  black_detect = black_detect*(255/np.max(black_detect))
  binary_black_detect = np.zeros_like(black_detect)  
  binary_black_detect[black_detect > 0] = 1


  Lab_img = cv2.cvtColor(morph_simple_img, cv2.COLOR_RGB2Lab)

  L = Lab_img[:,:,0]  

  thresh_L = (0, 10)
  L = L*(255/np.max(L))
  binary_L = np.zeros_like(L)
  binary_L[(L > thresh_L[0]) & (L <= thresh_L[1])] = 1

  #DEBUG
  #plt.figure(figsize = (20,10))
  #plt.imshow(binary_L, cmap=plt.cm.gray, interpolation='nearest')
    
  cont_merge = np.zeros_like(L) 
  cont_merge[(binary_L == 0) | (binary_black_detect == 1)] = 1

  #DEBUG
  #plt.figure(figsize = (20,10))
  #plt.imshow(cont_merge, cmap=plt.cm.gray, interpolation='nearest')

  contours_merge, hierarchy_merge = cv2.findContours(np.uint8(cont_merge), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  # Draw contours
  drawing_merge = np.zeros((binary_L.shape[0], binary_L.shape[1], 3), dtype=np.uint8)
  contours_merge = sorted(contours_merge, key=cv2.contourArea, reverse=True)

  for index, el in enumerate(contours_merge):
    if cv2.contourArea(el) > 20000:
        cv2.drawContours(drawing_merge, contours_merge, index, 255, thickness=-1) 
      
  binary_merge_Filtered = cv2.cvtColor(drawing_merge, cv2.COLOR_RGB2GRAY)

  #DEBUG
  #plt.figure(figsize = (20,10))
  #plt.imshow(binary_merge_Filtered, cmap=plt.cm.gray, interpolation='nearest')

  return binary_merge_Filtered

In [0]:
def detect_outliers(x,y,w,h,orizontal_dim,vertical_dim):

  ret = False
  general_threshold = 3.5
  border_threshold  = 2.5

  if w > h:
    ratio = w/h
  else:
    ratio = h/w

  #print(ratio)

  if ratio >= general_threshold:
    ret = True  

  if ret == False:
    if (w*h) < (0.005*(orizontal_dim*vertical_dim)):
        ret = True

  return ret

# **SVM Classifications**

In [0]:
def people_classification(warped_image, people_statue_model):
        
    valid_match = 0

    orient = 44
    pix_per_cell = 8
    cell_per_block = 1
    
    warped_image = cv2.resize(warped_image, (100,100), interpolation = cv2.INTER_AREA) 
    warped_image = cv2.cvtColor(warped_image, cv2.COLOR_RGB2HLS)
    test_hog_features = []
    for channel in range(warped_image.shape[2]):
        test_hog_features.append(get_hog_features(warped_image[:,:,channel], orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True))
    test_hog_features = np.ravel(test_hog_features)

    
    #print(people_statue_model.predict_proba(test_hog_features.reshape(1,-1))[0][1])
    if people_statue_model.predict_proba(test_hog_features.reshape(1,-1))[0][1] > 0.57:
        valid_match = 1
    else:
        valid_match = 0

    return  valid_match  

In [0]:
def painting_classification(image, model):

    valid_match = 0

    orient = 9
    pix_per_cell = 8
    cell_per_block = 2
    
    image = cv2.resize(image, (128,128), interpolation = cv2.INTER_AREA)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2Lab)

    test_hog_features = []
    for channel in range(image.shape[2]):
        test_hog_features.append(get_hog_features(image[:,:,channel], orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True))
    test_hog_features = np.ravel(test_hog_features)

    valid_match = model.predict_proba(test_hog_features.reshape(1,-1))[0][1]
    #print(valid_match)

    return valid_match

# **SIFT Classification**

In [0]:
def sift_classification_single_img(warped_image, model, matrix_db, names_db, topn=6, nfeatures=500):

  prediction = 0
  valid_match = 0
  sift =  cv2.xfeatures2d.SIFT_create(nfeatures) 

  # FLANN matcher
  FLANN_INDEX_KDTREE = 0
  index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
  search_params = dict(checks=50)  
  flann = cv2.FlannBasedMatcher(index_params,search_params)

  matches_flann = []   
  
  kp_test, des_test = sift.detectAndCompute(cv2.cvtColor(warped_image,cv2.COLOR_RGB2GRAY), None)
  for vector in range(len(matrix_db)):
    if type(matrix_db[vector]) is np.ndarray and type(des_test) is np.ndarray:
      matches_count = 0 
      if len(kp_test) >= 2:
          matches = flann.knnMatch(matrix_db[vector], des_test,k=2)
          if len(matches[vector]) == 2:
            for x,(m,n) in enumerate(matches):
              if m.distance < 0.7*n.distance:
                  matches_count += 1
            matches_flann.append((matches_count,names_db[vector]))
  if len(matches_flann) > 2:    
    matches_flann.sort(key=lambda x : x[0] , reverse = True)
    prediction = matches_flann[:topn]
    valid_match = 1

  return prediction, valid_match

# **Paintings Detection**

In [0]:
def detect_painting(countour_simple_img, source_img, output_mode, model, matrix_db, names_db):

  
  contours, hierarchy = cv2.findContours(np.uint8(countour_simple_img), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  # Draw contours
  drawing = np.zeros((countour_simple_img.shape[0], countour_simple_img.shape[1], 3), dtype=np.uint8)
  contours = sorted(contours, key=cv2.contourArea, reverse=True)

  centres = []
  area = []
  for index, el in enumerate(contours):
    if index == 0:
      biggest_area = cv2.contourArea(el)
    if cv2.contourArea(el) > 20000:
        cv2.drawContours(drawing, contours, index, 255, thickness=-1) 
        moments = cv2.moments(contours[index])
        centres.append((int(moments['m10']/moments['m00']), int(moments['m01']/moments['m00'])))
        area.append(cv2.contourArea(el))
        cv2.circle(drawing, centres[-1], 6, (0, 255, 0), -1)
       

  midpoint = []
  for ref_idx in range(len(centres)):
    min_dist = countour_simple_img.shape[0] + countour_simple_img.shape[1]
    for comp_idx in range(len(centres)):  
         measure = dist.euclidean((centres[ref_idx][0] , centres[ref_idx][1]), (centres[comp_idx][0] , centres[comp_idx][1]))
         if measure > 0:
          if measure < min_dist:
            min_dist = measure

    min_dist = int(min_dist/2)
    midpoint.append(min_dist) 
  
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4,4))
  drawing_bin = cv2.cvtColor(drawing, cv2.COLOR_RGB2GRAY)
  drawing_bin = cv2.erode(drawing_bin, kernel, iterations = 8)
  drawing_bin = cv2.dilate(drawing_bin, kernel, iterations = 2)
  drawing_bin = cv2.erode(drawing_bin, kernel,iterations = 2)

  #DEBUG
  #plt.figure(figsize = (20,10))
  #plt.imshow(drawing, cmap=plt.cm.gray, interpolation='nearest')

  out = np.zeros_like(source_img)
  offset_x = 0
  offset_y = 0   
  past_coord = [(1,1,1,1)]
  first_attempt = False 
  warped_img = []
  predictions = []
  boxes = []
    
  for centre_idx in range(len(centres)):
    
    sec_margin = 0
    coeff = int((0.0005/(centre_idx+1))*area[centre_idx])

    if area[centre_idx] <= 0.5*biggest_area:
      coeff = int((0.0001/(centre_idx+1))*area[centre_idx])

    img_off_x2 = np.clip(centres[centre_idx][0] + int(coeff + midpoint[centre_idx]) + sec_margin, 0 , drawing_bin.shape[1])
    img_off_x1 = np.clip(centres[centre_idx][0] - int(coeff + midpoint[centre_idx]) - sec_margin, 0 , drawing_bin.shape[1])
    img_off_y2 = np.clip(centres[centre_idx][1] + int(coeff + midpoint[centre_idx]) + sec_margin, 0 , drawing_bin.shape[0])
    img_off_y1 = np.clip(centres[centre_idx][1] - int(coeff + midpoint[centre_idx]) - sec_margin, 0 , drawing_bin.shape[0])

    cropped = drawing_bin[img_off_y1:img_off_y2,img_off_x1:img_off_x2]
  

    #### DEBUG #####
    #plt.figure(figsize = (20,10))
    #plt.imshow(cropped, cmap=plt.cm.gray, interpolation='nearest') 

    offset_x = 0
    if img_off_x1 > 0:
          offset_x = img_off_x1 

    offset_y = 0
    if img_off_y1 > 0:
          offset_y = img_off_y1     
    
    countours_2, hierarchy = cv2.findContours(cropped, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    countours_2 = sorted(countours_2, key=cv2.contourArea, reverse=True)
   
    for idx, el in enumerate(countours_2):   
       
      if cv2.contourArea(el) > 10000 and idx == 0: 

        epsilon = 0.0001 * cv2.arcLength(countours_2[0], True)
        approx = cv2.approxPolyDP(countours_2[0], epsilon, True)
        
        
        t = 10        
        n = approx.shape[0]

        for i in range(n): 

            p0 = approx[(i+n-1) % n][0]    
            p1 = approx[i][0]              
            p2 = approx[(i + 1) % n][0]     
            dx = (p2[0] + offset_x)-(p1[0] + offset_x)  
            dy = (p2[1] + offset_y)-(p1[1] + offset_y)  
                      
            
            if abs(dx) < t:
                if ((dx < 0) and (p0[0] > p1[0])) or ((dx > 0) and (p0[0] < p1[0])):
                    p2[0] = p1[0]
                else:
                    p1[0] = p2[0]

            
            if abs(dy) < t:
                if ((dy < 0) and (p0[1] > p1[1])) or ((dy > 0) and (p0[1] < p1[1])):
                    p2[1] = p1[1]
                else:
                    p1[1] = p2[1]
                    
            approx[i][0] = p1
            approx[(i + 1) % n][0] = p2
            
            
        #### DEBUG #####
        #dummy = np.zeros((cropped.shape[0], cropped.shape[1], 3), dtype=np.uint8)
        #cv2.drawContours(dummy,approx,-1,(0,255,0),10)        
        #x,y,w,h = cv2.boundingRect(el)
        #cv2.rectangle(dummy,(x,y),(x+w,y+h),(0,0,255),2)
        #plt.figure(figsize = (20,10))
        #plt.imshow(dummy, cmap=plt.cm.gray, interpolation='nearest')

        x_crop,y_crop,w_crop,h_crop = cv2.boundingRect(el)
        mask = np.zeros((source_img.shape[0], source_img.shape[1]))
        painting_corners_raw = retrieve_points_coordinates(cv2.boundingRect(el), offset_x, offset_y)              
        painting_corners = painting_corners_raw.reshape(4,1,2)

        if len(approx) < 4:
          cv2.fillConvexPoly(0, painting_corners, 1)   
        else:     
          x,y,w,h = cv2.boundingRect(painting_corners)

 
          if detect_outliers(x,y,w,h,source_img.shape[1],source_img.shape[0]) == False:

            no_shadowing = False
            for shape in range(len(past_coord)):
              
              if  (((x >= past_coord[shape][0]) and (x <= (past_coord[shape][0] + past_coord[shape][2]))) and ((y >= past_coord[shape][1]) and (y <= (past_coord[shape][1] + past_coord[shape][3]))))==False\
                  and\
                  ((((x + w) >= past_coord[shape][0]) and ((x + w) <= (past_coord[shape][0] + past_coord[shape][2]))) and (((y + h) >= past_coord[shape][1]) and ((y + h) <= (past_coord[shape][1] + past_coord[shape][3]))))==False:                    

                    if first_attempt == False:   
                      past_coord[shape] = (x,y,w,h)
                      first_attempt =True  
                    else:                     
                      past_coord.append((x,y,w,h)) 
                    if no_shadowing == False:
                       no_shadowing = True
              else:
                 no_shadowing = False
                 break
 
            if no_shadowing == True:                
              median = x+(w/2)                      
              painting_corners_warp = order_points_by_clusters(approx.reshape(-2,2), median, offset_x, offset_y)  
              #painting_corners_warp = order_points(approx.reshape(-2,2), offset_x, offset_y) 
              valid_match = painting_classification(source_img[y_crop+offset_y:y_crop+h_crop+offset_y,x_crop+offset_x:x_crop+w_crop+offset_x], model)  
              if valid_match > 0.65:
                prediction, matches = sift_classification_single_img(perspective_transform(source_img, painting_corners_warp), model, matrix_db, names_db)
                if matches > 0:
                  warped_img.append(perspective_transform(source_img, painting_corners_warp))
                  predictions.append((len(warped_img), prediction))

                  cv2.fillConvexPoly(mask, painting_corners_warp, 1)              
                  #cv2.fillConvexPoly(mask, sum_offset_coordinates(approx.reshape(-2,2), offset_x, offset_y), 1) 
                  mask = mask.astype(np.bool)
                  out[mask] = source_img[mask]    
                  #plt.figure(figsize = (20,10))
                  #plt.imshow(out[mask], cmap=plt.cm.gray, interpolation='nearest')  

                  #cv2.rectangle(source_img,(x,y),(x+w,y+h),(0,255,0),6) 
                  boxes.append((x,y,x+w,y+h))                    
                  #print('BOX: ' + str((x,y,x+w,y+h))) 

  overlap = cv2.addWeighted(source_img,0.2,out,0.7,0)

  if output_mode == 1:
    return source_img,warped_img,predictions,boxes
  elif output_mode == 2:
    return out,warped_img,predictions,boxes
  elif output_mode == 3:
    return overlap,warped_img,predictions,boxes 

In [0]:
def non_maxima_suppression(start_x1, start_y1, end_x1, end_y1, start_x2, start_y2, end_x2, end_y2):

  proceed = False

  if  (((start_x1 >= start_x2) and (start_x1 <= (end_x2))) and ((start_y1 >= start_y2) and (start_y1 <= (end_y2))))==False\
      and\
      ((((end_x1) >= start_x2) and ((end_x1) <= (end_x2))) and (((end_y1) >= start_y2) and ((end_y1) <= (end_y2))))==False:  

      proceed = True

  return proceed    

In [0]:
def discard_cameras(start_y1, frame):

  proceed = False
  
  if start_y1 > (frame.shape[0]/3):
     proceed = True

  return proceed 

# **People Detection**

In [0]:
def people_detection(source_img, weightsPath, configPath, external_boxes, people_statue_model):

  visualize = 0
  human_presence = 0
  threshold = 0.5
  confidence_thr = 0.57

  people_boxes = []
  statues_boxes = []
  
  net = cv2.dnn.readNetFromTensorflow(weightsPath, configPath)

  clone = source_img.copy()
  image = source_img.copy()

  (H, W) = image.shape[:2]

  blob = cv2.dnn.blobFromImage(image, swapRB=True, crop=False)
  net.setInput(blob)
  start = time.time()
  (boxes, masks) = net.forward(["detection_out_final", "detection_masks"])
  end = time.time()
  # loop over the number of detected objects
  for i in range(0, boxes.shape[2]):
 
    classID = int(boxes[0, 0, i, 1])    
    confidence = boxes[0, 0, i, 2]

    #Only People 
    if classID == 0:
      if confidence > confidence_thr:       
        box = boxes[0, 0, i, 3:7] * np.array([W, H, W, H])
        (startX, startY, endX, endY) = box.astype("int")

        no_outliers = True
        
        for i in range(len(external_boxes)):
          no_outliers = non_maxima_suppression(startX, startY, endX, endY, external_boxes[i][0],external_boxes[i][1],external_boxes[i][2],external_boxes[i][3])          
          
          if no_outliers == False:
            break

          no_outliers = discard_cameras(startY, source_img) 

          if no_outliers == False:
            break 

        if no_outliers == True:

          boxW = endX - startX
          boxH = endY - startY
         
          mask = masks[i, classID]
          mask = cv2.resize(mask, (boxW, boxH),interpolation=cv2.INTER_NEAREST)
          mask = (mask > threshold)

          roi = clone[startY:endY, startX:endX]
       
          if people_classification(roi, people_statue_model) > 0:

            roi = roi[mask]

            color = np.array([255,0,0])            
            blended = ((0.4 * color) + (0.6 * roi)).astype("uint8")

            clone[startY:endY, startX:endX][mask] = blended

            color = [int(c) for c in color]
            cv2.rectangle(clone, (startX, startY), (endX, endY), color, 2)
            people_boxes.append((startX,startY,endX,endY))

            text = "{}: {:.4f}".format(LABELS[classID], confidence)
            cv2.putText(clone, text, (startX, startY - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)
            human_presence = 1
        
          else:
            roi = roi[mask]            
            
            color = np.array([0,0,255])  
            blended = ((0.4 * color) + (0.6 * roi)).astype("uint8")
            
            clone[startY:endY, startX:endX][mask] = blended
            color = [int(c) for c in color] 
            cv2.rectangle(clone, (startX, startY), (endX, endY), color, 2)
            statues_boxes.append((startX,startY,endX,endY))

            text = 'statue'
            cv2.putText(clone, text, (startX, startY - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

  return clone, human_presence, people_boxes, statues_boxes 

In [0]:
def print_bounding_boxes(source_img, boxes):
  for i in range(len(boxes)):
    cv2.rectangle(source_img,(boxes[i][0],boxes[i][1]),(boxes[i][2],boxes[i][3]),(0,255,0),6) 
  return source_img

In [0]:
def resizeAndPad(img, size, padColor=0):

  h, w = img.shape[:2]
  sw, sh = size

  
  if h > sh or w > sw:
      interp = cv2.INTER_AREA

  else: 
      interp = cv2.INTER_CUBIC

  
  aspect = float(w)/h 
  saspect = float(sw)/sh

  if (saspect >= aspect) or ((saspect == 1) and (aspect <= 1)):  
      new_h = sh
      new_w = np.round(new_h * aspect).astype(int)
      pad_horz = float(sw - new_w) / 2
      pad_left, pad_right = np.floor(pad_horz).astype(int), np.ceil(pad_horz).astype(int)
      pad_top, pad_bot = 0, 0

  elif (saspect < aspect) or ((saspect == 1) and (aspect >= 1)): 
      new_w = sw
      new_h = np.round(float(new_w) / aspect).astype(int)
      pad_vert = float(sh - new_h) / 2
      pad_top, pad_bot = np.floor(pad_vert).astype(int), np.ceil(pad_vert).astype(int)
      pad_left, pad_right = 0, 0

  
  if len(img.shape) is 3 and not isinstance(padColor, (list, tuple, np.ndarray)): 
      padColor = [padColor]*3

  
  scaled_img = cv2.resize(img, (new_w, new_h), interpolation=interp)
  scaled_img = cv2.copyMakeBorder(scaled_img, pad_top, pad_bot, pad_left, pad_right, borderType=cv2.BORDER_CONSTANT, value=padColor)

  return scaled_img

# **Image Collage**

In [0]:
def print_text_on_image(background, counter_img, predictions, num_predictions, ident, width_warp, shift_y,text = ''):

  for title in range(num_predictions):
      text += str(title+1) + ': '
      if len(predictions) > 0:
        text += DB_title.get(predictions[counter_img][1][title][1])
      space = 80
      bold = 1
      if title == 0:
        bold = 2
      cv2.putText(background, text, ( ident + width_warp  + 10, shift_y + (title+1)*space), cv2.FONT_HERSHEY_DUPLEX, 1.5, (200,255,155), bold, cv2.LINE_AA)
      text = ''


def image_collage(source_img, warp_img, predictions, human_presence, num_predictions=6):

  if source_img.shape[1] > source_img.shape[0]:
      w = int(1920)
      h = int(1080)
      n_slot_right = 2
      
      buffer_x = int(w/5)
      buffer_y = int(h/3) 
      vertical_dim = 2*h + buffer_y
      horizontal_dim = 2*w  + 2*buffer_x 

      width_warp = buffer_x 
      height_warp = int((h/2))
      dim_warp = (width_warp, height_warp)
      shift_x = 0
      shift_y = 0
  else:
      h = int(1920)
      w = int(1080)
      n_slot_right = 6
     
      buffer_x = int(w/2)
      buffer_y = int(h/5)
      vertical_dim = 2*w + 3*buffer_y
      horizontal_dim = 3*h 
      
      width_warp = buffer_x 
      height_warp = int((h/2))
      dim_warp = (width_warp, height_warp)
      shift_x = width_warp
      shift_y = 0

  dim = (w, h) 
  source_img = cv2.resize(source_img, dim, interpolation = cv2.INTER_AREA)
  background = np.zeros((vertical_dim, horizontal_dim,3), np.uint8)
 
  background[:source_img.shape[0],:source_img.shape[1],:3] = source_img
 
  offset_x = width_warp
  offset_y = height_warp
  counter_img = 0

  
  for counter_img, img in enumerate(warp_img): 

    if counter_img < 6:

      resized = resizeAndPad(img, dim_warp, 200)

      if n_slot_right == 2:

        if counter_img < n_slot_right:
          background[shift_y:(shift_y + offset_y), (int(horizontal_dim/2)): (int(horizontal_dim/2)) + width_warp,:3] = resized 
          print_text_on_image(background, counter_img, predictions, num_predictions, int(horizontal_dim/2), width_warp, shift_y) 
          shift_y += offset_y
        else:
          if counter_img%2 != 0:
            background[buffer_y + int(buffer_y/2) + (int(vertical_dim/2)): buffer_y + int(buffer_y/2) + (int(vertical_dim/2)) + height_warp, shift_x:(shift_x+offset_x),:3] = resized
            print_text_on_image(background, counter_img, predictions, num_predictions, shift_x, width_warp,  buffer_y + int(buffer_y/2) + (int(vertical_dim/2)))
            shift_x += 6*offset_x
          else:
            background[(int(vertical_dim/2)): (int(vertical_dim/2)) + height_warp, shift_x:(shift_x+offset_x),:3] = resized
            print_text_on_image(background, counter_img, predictions, num_predictions, shift_x, width_warp, (int(vertical_dim/2))) 
                    
      elif n_slot_right == 6:
        if counter_img%2 != 0:          
          background[shift_y:(shift_y + offset_y), 2*shift_x + int(shift_x/2): 2*shift_x + int(shift_x/2) + width_warp,:3] = resized 
          print_text_on_image(background, counter_img, predictions, num_predictions, 2*shift_x + int(shift_x/2), width_warp,  height_warp)
          shift_x = offset_x + int(1.5*offset_x)
          shift_y = 0
          if counter_img > 3:
            shift_y = 4*buffer_y + int(buffer_y/2)   
        else:            
          background[shift_y:(shift_y + offset_y), 2*shift_x + int(shift_x/2): 2*shift_x + int(shift_x/2) + width_warp,:3] = resized
          print_text_on_image(background, counter_img, predictions, num_predictions, 2*shift_x + int(shift_x/2), width_warp, shift_y)
          shift_y += 2*buffer_y + int(buffer_y/2)
          if counter_img > 3:
             shift_y += 4*buffer_y + int(buffer_y/2)

  
  if human_presence > 0 and len(predictions) > 0:
    room = "Room : " + str(DB_room.get(predictions[0][1][0][1]))
    cv2.putText(background, room, (10, 100), cv2.FONT_HERSHEY_DUPLEX, 3, (0,150,255), 2, cv2.LINE_AA)

  return background

# **Image processing pipeline**

In [0]:
def image_processing_pipeline(img, weightsPath, configPath, paint_model, people_statue_model, matrix_db, names_db):

  source_img = img.copy().astype(np.uint8)

  countour_simple_img = transform_image(source_img)
  #DEBUG
  #plt.figure(figsize = (20,10))
  #plt.imshow(countour_simple_img, cmap=plt.cm.gray, interpolation='nearest')

  #Paintings Detection
  output_mode = 1
  detect_img, warp_img, predictions, painting_boxes = detect_painting(countour_simple_img, source_img, output_mode, paint_model, matrix_db, names_db)

  #People/Statues Detection
  people_img, human_presence, people_boxes, statues_boxes = people_detection(detect_img, weightsPath, configPath, painting_boxes, people_statue_model)

  #Print Paintings Bounding Boxes
  boxes_img = print_bounding_boxes(people_img, painting_boxes)

  #Final visualization
  out_img = image_collage(boxes_img, warp_img, predictions, human_presence)

  return out_img

# **Release Video**

In [72]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

class ProcessImage:
    def __init__(self, arg1, arg2, arg3, arg4, arg5, arg6):
        self.arg1 = arg1
        self.arg2 = arg2
        self.arg3 = arg3
        self.arg4 = arg4
        self.arg5 = arg5
        self.arg6 = arg6
    def __call__(self, image):
        out = image_processing_pipeline(image, self.arg1, self.arg2, self.arg3, self.arg4, self.arg5, self.arg6)
        return out

out_video = root_path + 'OutVideos/IMG_4077_Processed.MP4'
clips = VideoFileClip(root_path + 'Videos/IMG_4077.MP4')
final_clip = clips.fl_image(ProcessImage(weightsPath, configPath, paint_model, people_statue_model, matrix_db, names_db))
#final_clip.set_duration(10).write_videofile(out_video, audio=False) #-->> for shorter videos, ex: 10 seconds duration
final_clip.write_videofile(out_video, audio=False)

[MoviePy] >>>> Building video /content/drive/My Drive/Unimore/VCS/Project/OutVideos/IMG_4077_Processed.MP4
[MoviePy] Writing video /content/drive/My Drive/Unimore/VCS/Project/OutVideos/IMG_4077_Processed.MP4


100%|█████████▉| 645/646 [2:13:53<00:12, 12.46s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/drive/My Drive/Unimore/VCS/Project/OutVideos/IMG_4077_Processed.MP4 



# **Looping over video frames**

In [0]:
cam = cv2.VideoCapture(root_path + "Videos/IMG_4077.MP4") 
  
try: 
      
    # creating a folder named data 
    if not os.path.exists(root_path + 'Videos/Images'): 
        os.makedirs(root_path + 'Videos/Images') 
  
# if not created then raise error 
except OSError: 
    print ('Error: Creating directory of data') 
  
# frame 
currentframe = 0
# Number of frame 
n_frame = 0
  
while(True): 
      
    # reading from video 
    ret,frame = cam.read() 
  
    if ret: 

        currentframe += 1  

        if currentframe%10 == 0: # in case you want to skip some frames acting here, please
          
          converted = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
          out_img = image_processing_pipeline(converted, weightsPath, configPath, paint_model, people_statue_model, matrix_db, names_db)
 
          name = root_path + 'Videos/Images/frame' + str(currentframe) + '.jpg'
          print ('Creating...' + name) 

          cv2.imwrite(name, cv2.cvtColor(out_img, cv2.COLOR_BGR2RGB))
         
    else: 
        break
  
# Release all space and windows once done 
cam.release() 